In [1]:
### Base ###
import os
import numpy as np 
import torch 
import torch.nn as nn
from torch.optim import Adam
import fnmatch
from torch.utils.data import TensorDataset, DataLoader
import itertools
import math
from sklearn.decomposition import PCA
from scipy.ndimage import gaussian_filter
import PIL.Image as pimg
import nibabel as nib
from multiprocessing import Pool

print(os.environ['CONDA_DEFAULT_ENV'])
# from skimage import data
# from skimage import exposure
# from skimage.exposure import match_histograms
import itk

### Visualization ###
#import seaborn as sns
#sns.set(color_codes=True)
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)
rc('font', **{'family':'serif','serif':['Palatino']})

/Users/alexandre.bone/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


deformetrica


ModuleNotFoundError: No module named 'itk'

# 1. Normalize images with mri_normalize

In [ ]:
args = []

path_to_dataset__in = '/Users/alexandre.bone/Workspace/2020_MICCAI/2_datasets/1_brats_2019'
path_to_dataset__out = '/Users/alexandre.bone/Workspace/2020_MICCAI/2_datasets/2_t1_normalized'
grades = ['HGG', 'LGG']
img_type = 't1'
slice_id = 77

if not os.path.isdir(path_to_dataset__out): 
    os.mkdir(path_to_dataset__out)

for t in [('1_training', 'train')]: 
    path_to_t = os.path.join(path_to_dataset__out, t[1])
    if not os.path.isdir(path_to_t): 
        os.mkdir(path_to_t)
    for grade in grades: 
        path_to_subjects = os.path.join(path_to_dataset__in, t[0], grade)
        subject_ids = [elt for elt in os.listdir(path_to_subjects) if elt[:5] == 'BraTS']
        for subject_id in subject_ids[:1]:
            path_to_file__in = os.path.join(path_to_dataset__in, t[0], grade, subject_id, subject_id + '_%s.nii.gz' % img_type)
            path_to_file__out = os.path.join(path_to_dataset__out, t[1], grade.lower() + '_' + subject_id[8:] + '_' + img_type)
            args.append((path_to_file__in, path_to_file__out + '.nii.gz'))


def launch(args):
    path_in, path_out = args
    cmd = 'mri_normalize %s %s > log' % (path_in, path_out)
    print(cmd)
    os.system(cmd)

# os.system('/applications/FreeSurfer/freesurfer-v6.0.0/SetUpFreeSurfer.sh &> /dev/null')
os.system('/Applications/freesurfer/SetUpFreeSurfer.sh &> /dev/null')
with Pool(os.cpu_count()) as pool:
    pool.map(launch, args)

# 2. Extract a slice and stock it in png format

In [ ]:
path_to_dataset = '/Users/alexandre.bone/Workspace/2020_MICCAI/2_datasets/1_brats_2019'
t = '1_training'
grade = 'HGG'
subject_id = 'BraTS19_CBICA_AVF_1'
img_type = 't1ce'

path_to_file = os.path.join(path_to_dataset, t, grade, subject_id, subject_id + '_%s.nii.gz' % img_type)
img = nib.load(path_to_file).get_data()
print(img.shape)

figsize = 5
f = plt.figure(figsize=(figsize, figsize))
plt.imshow(np.transpose(img[::-1, ::-1, 77]), cmap='gray')
plt.axis('off')
plt.show()

plt.hist(img.reshape(-1, 1))
plt.show()

f = plt.figure(figsize=(figsize, figsize))
plt.imshow(np.transpose(np.clip(img, 0, 500)[::-1, ::-1, 77]), cmap='gray')
plt.axis('off')
plt.show()

plt.hist(np.clip(img, 0, 500).reshape(-1, 1))
plt.show()

def sigmoid(x,mi, mx): return mi + (mx-mi)*(lambda t: (1+20**(-t+0.5))**(-1) )( (x-mi)/(mx-mi) )

f = plt.figure(figsize=(figsize, figsize))
plt.imshow(np.transpose(sigmoid(img, 0, 500)[::-1, ::-1, 77]), cmap='gray')
plt.axis('off')
plt.show()

plt.hist(sigmoid(img, 0, 500).reshape(-1, 1))
plt.show()

In [ ]:
path_to_dataset__in = '/Users/alexandre.bone/Workspace/2020_MICCAI/2_datasets/1_brats_2019'
path_to_dataset__out = '/Users/alexandre.bone/Workspace/2020_MICCAI/2_datasets/4_t1ce_slice'
grades = ['HGG', 'LGG']
img_type = 't1ce'
slice_id = 77
max_intensity_clip = 500

if not os.path.isdir(path_to_dataset__out): 
    os.mkdir(path_to_dataset__out)

# for t in [('1_training', 'train'), ('2_validation', 'test')]: 
for t in [('1_training', 'train')]: 
    path_to_t = os.path.join(path_to_dataset__out, t[1])
    if not os.path.isdir(path_to_t): 
        os.mkdir(path_to_t)
    for grade in grades: 
        path_to_subjects = os.path.join(path_to_dataset__in, t[0], grade)
        subject_ids = [elt for elt in os.listdir(path_to_subjects) if elt[:5] == 'BraTS']
        for subject_id in subject_ids:
            path_to_file__in = os.path.join(path_to_dataset__in, t[0], grade, subject_id, subject_id + '_%s.nii.gz' % img_type)
            path_to_file__out = os.path.join(path_to_dataset__out, t[1], grade.lower() + '_' + subject_id[8:] + '_' + img_type)
            img = nib.load(path_to_file__in).get_data()
            img = np.transpose(img[::-1, ::-1, slice_id])
# #             max_intensity = max(max_intensity, np.max(img))
#             if np.max(img) > threshold: 
#                 taboo_files.append(os.path.basename(path_to_file__out))
            img = np.clip(img, 0, max_intensity_clip)
#             maxes.append(np.max(img))
#             medians.append(np.median(img[img!=0]))
#             mads.append(np.median(np.abs(img[img!=0] - np.median(img[img!=0]))))
            tol = 1e-10
            img = (np.clip(img.astype('float32') / float(max_intensity_clip), tol, 1.0 - tol) * 255).astype('uint8')
            pimg.fromarray(img).save(path_to_file__out + '.png')
#             np.save(path_to_file__out, img + '.npy)

# print('max_intensity = %d' % max_intensity)
plt.hist(np.array(maxes), bins=20)
plt.show()
print('np.mean(np.array(medians)) = %f' % np.mean(np.array(medians)))
print('np.mean(np.array(mads)) = %f' % np.mean(np.array(mads)))
print('len(taboo_files) = %d over %d files in total' % 
      (len(taboo_files), len([elt for elt in os.listdir(path_to_t) if elt[1:3] == 'gg'])))

In [ ]:
path_to_dataset__in = '/Users/alexandre.bone/Workspace/2020_MICCAI/2_datasets/1_brats_2019'
path_to_dataset__out = '/Users/alexandre.bone/Workspace/2020_MICCAI/2_datasets/2_t1_slice'
grades = ['HGG', 'LGG']
img_type = 't1'
slice_id = 77

imgs = []
path_to_files_out = []

if not os.path.isdir(path_to_dataset__out): 
    os.mkdir(path_to_dataset__out)

# for t in [('1_training', 'train'), ('2_validation', 'test')]: 
for t in [('1_training', 'train')]: 
    path_to_t = os.path.join(path_to_dataset__out, t[1])
    if not os.path.isdir(path_to_t): 
        os.mkdir(path_to_t)
    for grade in grades: 
        path_to_subjects = os.path.join(path_to_dataset__in, t[0], grade)
        subject_ids = [elt for elt in os.listdir(path_to_subjects) if elt[:5] == 'BraTS']
        for subject_id in subject_ids:
            path_to_file__in = os.path.join(path_to_dataset__in, t[0], grade, subject_id, subject_id + '_%s.nii.gz' % img_type)
            img = nib.load(path_to_file__in).get_data()
            img = np.transpose(img[::-1, ::-1, slice_id])
            imgs.append(imgs)
            path_to_files_out.append(os.path.join(path_to_dataset__out, t[1], grade.lower() + '_' + subject_id[8:] + '_' + img_type))
            
imgs = np.array(imgs)
imgs = (imgs - np.mean(imgs)) / np.std(imgs)

for (path, img) in zip(path_to_files_out, imgs): 
    img = (np.clip(img.astype('float32') / float(normalizer), tol, 1.0 - tol) * 255).astype('uint8')
    pimg.fromarray(img).save(path_to_file__out + '.png')

In [ ]:
np.median(img)

In [ ]:
import itk